In [ ]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os

### Extracting features

In [ ]:
training_dataset = "dataset/training"

In [ ]:
# get paths of each file in folder named Images
# Images here contains my data(folders of various persons)

imagePaths = list(paths.list_images(training_dataset))
knownEncodings = []
knownNames = []

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    name = imagePath.split(os.path.sep)[-2]
    # load the input image and convert it from BGR (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)

# save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}
# use pickle to save data into a file for later use
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

### Face Recognition

In [ ]:
testing_dataset = 'dataset/testing'

In [ ]:
# find path of xml file containing haarcascade file
cascPathface = os.path.dirname(
 cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved earlier
data = pickle.loads(open('face_enc', "rb").read())

In [ ]:
def identify_face(image):
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #convert image to Greyscale for haarcascade
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    encodings = face_recognition.face_encodings(rgb)

    if len(encodings) > 1:
        print("Found more than one encoding. Most likely, the image has multiple faces")
        return None, None
    
    names = []
    matches = face_recognition.compare_faces(data["encodings"], encodings[0])

    if True in matches:
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            #Check the names at respective indexes we stored in matchedIdxs
            name = data["names"][i]
            #increase count for the name we got
            counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)


        # update the list of names
        names.append(name)
        if len(names) > 1:
            print("Found more than one match for facial encoding")
            return None, None
        
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 2)
    else:
        print("No encoding matches found. Possibly a new face")
        return None, None
#     cv2.imshow("Frame", image)
#     cv2.waitKey(0)
    if len(names) == 0:
        return None
    return names[0], image

In [ ]:
testing_mapping = {0: 'Arnold_Schwarzenegger', 
                   1: 'Donald_Rumsfeld', 
                   2: 'George_W_Bush', 
                   3: 'Gerhard_Schroeder', 
                   4: 'Hugo_Chavez', 
                   5: 'Jacques_Chirac', 
                   6: 'Tony_Blair', 
                   7: 'Vladimir_Putin'}

In [ ]:
imagePaths = list(paths.list_images(testing_dataset))

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    try:
        name = testing_mapping[int(imagePath.split(os.path.sep)[-2])]
    except ValueError:
        name = "Unknown"
    image = cv2.imread(imagePath)
    pred_name, pred_image = identify_face(image)
    if pred_name is None:
        print(f"Face could not be recognized for {imagePath.split(os.path.sep)[-1]}\n")
    else:
        print(f"Original face: {name}; Identified face: {pred_name}\n")
#     cv2.imshow("Frame", pred_image)
#     cv2.waitKey(0)